# sources

https://github.com/openai/CLIP/blob/fcab8b6eb92af684e7ff0a904464be7b99b49b88/notebooks/Prompt_Engineering_for_ImageNet.ipynb
https://github.com/openai/CLIP#zero-shot-prediction

# Install the clip package and its dependencies


In [ ]:
# remove # to download packages

#! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-9hxn110e
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-9hxn110e
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 1.3 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369500 sha256=e973bef6e37a4fd8bbaea5218355021195bf909318c7e8581c02411551bf834a
  Stored in directory: /tmp/pip-ephem-wheel-cache-t3tarvz3/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.10
    Uninstalling wcwidth-0.2.10:
      Successfully uninstalled wcwidth-0.2.10


In [ ]:
# Import packages

import os
import numpy as np
import torch
import clip
from tqdm.notebook import tqdm
from pkg_resources import packaging
from torch.utils.data import Subset

In [ ]:
# available CLIP models
# In this experiment, ViT-L/14@336px is used, as this is also the default model in the original CLIP paper
print(clip.available_models())

# Loading in model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-L/14@336px', device)

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']


100%|███████████████████████████████████████| 891M/891M [00:11<00:00, 83.4MiB/s]


In [ ]:
# Parameters model

input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 427,944,193
Input resolution: 336
Context length: 77
Vocab size: 49408


# MNIST

In [ ]:
# Import MNIST dataset

from torchvision.datasets import  MNIST


In [ ]:
# Loading in MNIST train set
# Make sure train is set to true
# Train set contains 60.000 images
mnist_train = MNIST(root=os.path.expanduser("~/.cache"), download=True, train=True)

# Loading in MNIST test set
# Test set contains 10.000 images
mnist_test = MNIST(root=os.path.expanduser("~/.cache"), download=True, train=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 99372847.34it/s]


Extracting /root/.cache/MNIST/raw/train-images-idx3-ubyte.gz to /root/.cache/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 24711483.85it/s]


Extracting /root/.cache/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.cache/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 26840631.81it/s]


Extracting /root/.cache/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.cache/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3562178.15it/s]


Extracting /root/.cache/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.cache/MNIST/raw



In [ ]:
# Inspecting number of data points in each set
# Check whether all train and test images are loaded in

print(mnist_train)
print(mnist_test)

Dataset MNIST
    Number of datapoints: 60000
    Root location: /root/.cache
    Split: Train
Dataset MNIST
    Number of datapoints: 10000
    Root location: /root/.cache
    Split: Test


In [ ]:
# Takes long to run code on 60.000 images
# To test run code on smaller batch, subset of training data is created, containing 10% of the original dataset
# Subset of training dataset (10%, or 6000 0f the 60.000 datsets )

subset_mnist_train = Subset(mnist_train, indices=range(len(mnist_train) // 10))

# Creating image and word embeddings

In [ ]:
# Defining classes
mnist_classes = ['0','1','2','3','4','5','6','7','8','9',]
class_map = {'mnist': mnist_classes}

# Defining prompts
# Experiment with different prompts, as the entire prompt is encoded
# and will influence how similar the image and text embedding are
mnist_templates = ['a photo of the number: "{}".',]
template_map = {'mnist': mnist_templates}

In [ ]:
# Function to calculate accuracy

@torch.no_grad()
def accuracy(output, target, topk=(1,)):
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.reshape(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)

        res.append(correct_k.mul_(100.0 / batch_size).item())
    return res

In [ ]:
# Function to create text embeddings

@torch.no_grad()
def extract_text_features(dataset_name):
    class_names = class_map[dataset_name]
    templates = template_map[dataset_name]
    model.to(device)
    model.eval()

    zeroshot_weights = []
    for classname in class_names:
        texts = [template.format(classname) for template in templates]
        texts = clip.tokenize(texts).to(device)
        class_embeddings = model.encode_text(texts)
        # embedding size = 768,  (= dimensionality of the encoder layers)
        # each template + class gets vector of length 768
        # size of tensor is (x, 768), x refers to the number of templates included
        # x = rows, 768 = columns
        class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
        # normalized over rows (Frobenius Norm)
        # dimension remains (x, 768)
        class_embedding = class_embeddings.mean(dim=0)
        # Column wise mean is calculated
        # Vector of length 768 remains
        class_embedding /= class_embedding.norm()
        # vector is normalized (Frobenius normalization)
        zeroshot_weights.append(class_embedding)
        # every input (class + every possible template => mean) is represented as embedding of length 768
        # In this case, list of 10 embeddings with length 768
    zeroshot_weights = torch.stack(zeroshot_weights, dim=1).to(device)
    # list of embeddings is concatenated in matrix
    # embeddings are joined on column level

    torch.set_printoptions(profile="full")
    return zeroshot_weights

In [ ]:
image_features = []
image_labels = []
# !! Loading in the data: images + ground truth label (HAVE TO BE LOADED IN IN THE SAME ORDER)
for image, class_id in subset_mnist_train:
    # Preprocess: pixel intensity is normalized using dataset mean and standard deviation.
    # Input images are resized and center-cropped to conform with the expected image resolution.
    image_input = preprocess(image).unsqueeze(0).to(device)
    # unsqueeze(x), returns a new tensor with a dimension of size one inserted at the specified position.
    # x refers to the index at which to insert the singleton dimension

    with torch.no_grad():
        image_feature = model.encode_image(image_input)
        # matrix with dimenension (1, 768)

    image_feature /= image_feature.norm()
    # image features are normalized, if dim is None, the norm is calculated across all dimensions of input

    image_features.append(image_feature)
    # list of embedding (length 768) for each image
    image_labels.append(class_id)
    # Class id (the factorized ground truth) is added in a list of tensors

In [ ]:
image_features = torch.stack(image_features, dim=-1).to(device)
# separate embeddings are concatenated on the second of 2 dimensions, (so on rows)
# The final matrix has the shape [768, 10]

# Squeeze to get a tensor returned with all the dimensions of input of size 1 removed
image_features = image_features.squeeze()

In [ ]:
# extract text feature
text_features = extract_text_features('mnist')

In [ ]:
# compute top-1 accuracy
logits = (100. * image_features.T @ text_features).softmax(dim=-1)
# image features are multiplied by text features, softmax converts real numbers into probability distribution
# Output tensor lies in range 0-1 and sums to 1
image_labels = torch.tensor(image_labels).unsqueeze(dim=1).to(device)

top1_acc = accuracy(logits, image_labels, (1,))

print(f'top-1 accuracy for = dataset: {top1_acc[0]:.3f}')

top-1 accuracy for = dataset: 81.400
